In [3]:
## 1) Imports

import csv
import glob
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt

In [4]:
## 2) Helpers (file iteration + merge)

def iter_input_files(pattern: str):
    for filename in sorted(glob.glob(pattern)):
        path = Path(filename)
        if path.is_file():
            yield path


def merge_files(input_pattern: str, output_path: Path) -> None:
    with output_path.open("w", newline="") as output_file:
        writer = csv.writer(output_file)
        writer.writerow(["date", "time", "flow_rate_L_min", "volume"])

        for path in iter_input_files(input_pattern):
            with path.open(newline="") as input_file:
                reader = csv.reader(input_file)
                for row in reader:
                    if len(row) < 3:
                        continue
                    try:
                        timestamp = float(row[0])
                    except ValueError:
                        continue
                    dt = datetime.fromtimestamp(timestamp)
                    writer.writerow([
                        dt.date().isoformat(),
                        dt.time().replace(microsecond=0).isoformat(),
                        row[1],
                        row[2],
                    ])

In [5]:
## 3) Helpers (load series + plotting)

def load_series(path: Path):
    timestamps = []
    flow_rates = []
    volumes = []
    with path.open(newline="") as input_file:
        reader = csv.reader(input_file)
        for row in reader:
            if len(row) < 3:
                continue
            try:
                timestamp = float(row[0])
                flow_rate = float(row[1])
                volume = float(row[2])
            except ValueError:
                continue
            timestamps.append(datetime.fromtimestamp(timestamp))
            flow_rates.append(flow_rate)
            volumes.append(volume)
    return timestamps, flow_rates, volumes


def save_plot(x, y, title: str, ylabel: str, output_path: Path) -> None:
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(x, y, linewidth=1)
    ax.set_title(title)
    ax.set_xlabel("Time")
    ax.set_ylabel(ylabel)
    ax.grid(True, alpha=0.3)
    fig.autofmt_xdate()
    output_path.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(output_path, dpi=150, bbox_inches="tight")
    plt.close(fig)


In [6]:
## 4) Merge all daily CSVs into one

merge_files("Modbus_readings_*.csv", Path("merged_modbus_readings.csv"))
print("Merged files into merged_modbus_readings.csv")

Merged files into merged_modbus_readings.csv


In [7]:
## 5) Plot each day (time vs flow rate, time vs volume)

output_dir = Path("plots")
for path in iter_input_files("Modbus_readings_*.csv"):
    timestamps, flow_rates, volumes = load_series(path)
    if not timestamps:
        continue
    stem = path.stem
    save_plot(
        timestamps,
        flow_rates,
        title=f"{stem} - Time vs Flow Rate",
        ylabel="Flow Rate (L/min)",
        output_path=output_dir / f"{stem}_flow_rate.png",
    )
    save_plot(
        timestamps,
        volumes,
        title=f"{stem} - Time vs Volume",
        ylabel="Volume",
        output_path=output_dir / f"{stem}_volume.png",
    )
print(f"Saved plots to {output_dir}")


Saved plots to plots
